In [25]:
import re
from bs4 import BeautifulSoup

In [146]:
import re
from bs4 import BeautifulSoup

SECTION_PATTERN = "(\s*第[一二三四五六七八九十百千]+章\s+.*?\n)"
ARTICLE_PATTERN = "(\s*第[一二三四五六七八九十百千]+条)"

def process_html(content):
    # 解析 HTML 内容
    soup = BeautifulSoup(content, "html.parse")
    result = []
    rules = soup.find_all("p", class_="law-rule-text")
    current_article = None
    
    for rule in rules:
        num_span = rule.find("span", class_="law-rule-num")
        text_span = rule.find("span", class_="rule-text")
        if num_span and text_span:
            # 新条目
            if current_article:
                result.append(current_article)
            current_article = {"content": text_span.text.strip(), "law_section": None, "law_article": num_span.text.strip()}
        elif current_article:
            # 合并到当前条目
            current_article["content"] += "\n" + rule.text.strip()
    
    if current_article:
        result.append(current_article)
    
    return result

def process_string(content):
    content = content.replace("\u3000", " ")
    
    # 判断是否包含章节信息
    has_sections = bool(re.search(SECTION_PATTERN, content))
    result = []
    current_article = None
    current_section = None
        
    if has_sections:
        # 按章节和条文划分
        sections = re.split(SECTION_PATTERN, content)
        for section in sections:
            section_match = re.search(SECTION_PATTERN, section)
            if section_match:
                current_section = section_match.group().strip()
            # ['第一条', 'xxx', '第二条', 'xxx', ...]
            articles = re.split(ARTICLE_PATTERN, section)
            article_contents = []
            is_first_match = True
            print(f"Fuck {current_section} {articles}")
            # NOTE: manually add an auxiliary article so that all articles are processed by the same logic
            articles.append("第四条")
            for article in articles:
                article = article.strip()
                if len(article) == 0:
                    continue
                article_match = re.search(ARTICLE_PATTERN, article)
                # NOTE: the first article_match indicates an empty article_contents, ignore it
                if article_match:
                    if not is_first_match:
                        result.append({"content": "\n".join(article_contents), "law_section": current_section, "law_article": current_article})
                    article_contents.clear()
                    current_article = article_match.group().strip()
                    is_first_match = False
                # NOTE: split text to unify formats
                article_contents.extend([x for x in re.split("\s+", article.strip()) if len(x)])
                # print(f"Shit {article} article_contents {article_contents}")
    else:
        # 仅按条文划分
        # ['第一条', 'xxx', '第二条', 'xxx', ...]
        articles = re.split(ARTICLE_PATTERN, content)
        # NOTE: manually add an auxiliary article so that all articles are processed by the same logic
        articles.append("第四条")
        article_contents = []
        is_first_match = True
        # print(f"Fuck {current_section} {articles}")
        for article in articles:
            article = article.strip()
            if len(article) == 0:
                continue
            article_match = re.search(ARTICLE_PATTERN, article)
            if article_match:
                # NOTE: the first article_match indicates an empty article_contents, ignore it
                if not is_first_match:
                    result.append({"content": "\n".join(article_contents), "law_section": current_section, "law_article": current_article})
                article_contents.clear()
                current_article = article_match.group().strip()
                is_first_match = False
            # NOTE: split text to unify formats
            article_contents.extend([x for x in re.split("\s+", article.strip()) if len(x)])
            # print(f"Shit {article} article_contents {article_contents}")
    return result

In [147]:
# print(process_html(a))

string_with_section = """
中华人民共和国招标投标法

（1999年8月30日第九届全国人民代表大会常务委员会第十一次会议通过　根据2017年12月27日第十二届全国人民代表大会常务委员会第三十一次会议《关于修改<中华人民共和国招标投标法>、<中华人民共和国计量法>的决定》修正）

　　第一条　为了规范招标投标活动，保护国家利益、社会公共利益和招标投标活动当事人的合法权益，提高经济效益，保证项目质量，制定本法。

　　第二条　在中华人民共和国境内进行招标投标活动，适用本法。

　　第三条　在中华人民共和国境内进行下列工程建设项目包括项目的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料等的采购，必须进行招标:

　　（一）大型基础设施、公用事业等关系社会公共利益、公众安全的项目；

　　（二）全部或者部分使用国有资金投资或者国家融资的项目；

　　（三）使用国际组织或者外国政府贷款、援助资金的项目。

　　前款所列项目的具体范围和规模标准，由国务院发展计划部门会同国务院有关部门制订，报国务院批准。

　　法律或者国务院对必须进行招标的其他项目的范围有规定的，依照其规定。

　　第四条　任何单位和个人不得将依法必须进行招标的项目化整为零或者以其他任何方式规避招标。

　　第五条　招标投标活动应当遵循公开、公平、公正和诚实信用的原则。

　　第六条　依法必须进行招标的项目，其招标投标活动不受地区或者部门的限制。任何单位和个人不得违法限制或者排斥本地区、本系统以外的法人或者其他组织参加投标，不得以任何方式非法干涉招标投标活动。

　　第七条　招标投标活动及其当事人应当接受依法实施的监督。

　　有关行政监督部门依法对招标投标活动实施监督，依法查处招标投标活动中的违法行为。

　　对招标投标活动的行政监督及有关部门的具体职权划分，由国务院规定。
"""
res = process_string(string_with_section)

# string_no_section = """
# 第1条 本法适用于...
# 第2条 本法规定...
# """
# print(process_law(string_no_section))

In [148]:
res

[{'content': '第一条\n为了规范招标投标活动，保护国家利益、社会公共利益和招标投标活动当事人的合法权益，提高经济效益，保证项目质量，制定本法。',
  'law_section': None,
  'law_article': '第一条'},
 {'content': '第二条\n在中华人民共和国境内进行招标投标活动，适用本法。',
  'law_section': None,
  'law_article': '第二条'},
 {'content': '第三条\n在中华人民共和国境内进行下列工程建设项目包括项目的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料等的采购，必须进行招标:\n（一）大型基础设施、公用事业等关系社会公共利益、公众安全的项目；\n（二）全部或者部分使用国有资金投资或者国家融资的项目；\n（三）使用国际组织或者外国政府贷款、援助资金的项目。\n前款所列项目的具体范围和规模标准，由国务院发展计划部门会同国务院有关部门制订，报国务院批准。\n法律或者国务院对必须进行招标的其他项目的范围有规定的，依照其规定。',
  'law_section': None,
  'law_article': '第三条'},
 {'content': '第四条\n任何单位和个人不得将依法必须进行招标的项目化整为零或者以其他任何方式规避招标。',
  'law_section': None,
  'law_article': '第四条'},
 {'content': '第五条\n招标投标活动应当遵循公开、公平、公正和诚实信用的原则。',
  'law_section': None,
  'law_article': '第五条'},
 {'content': '第六条\n依法必须进行招标的项目，其招标投标活动不受地区或者部门的限制。任何单位和个人不得违法限制或者排斥本地区、本系统以外的法人或者其他组织参加投标，不得以任何方式非法干涉招标投标活动。',
  'law_section': None,
  'law_article': '第六条'},
 {'content': '第七条\n招标投标活动及其当事人应当接受依法实施的监督。\n有关行政监督部门依法对招标投标活动实施监督，依法查处招标投标活动中的违法行为。\n对招标投标活动的行政监督及有关部门的

In [79]:
re.split("(\s*第[一二三四五六七八九十百千]+章\s+.*?\n)", string_with_section)

['\n中华人民共和国招标投标法\n\n（1999年8月30日第九届全国人民代表大会常务委员会第十一次会议通过\u3000根据2017年12月27日第十二届全国人民代表大会常务委员会第三十一次会议《关于修改<中华人民共和国招标投标法>、<中华人民共和国计量法>的决定》修正）\n\n目\u3000\u3000录',
 '\n\n\u3000\u3000第一章\u3000总则\n',
 '',
 '\n\u3000\u3000第二章\u3000招标\n',
 '',
 '\n\u3000\u3000第三章\u3000投标\n',
 '',
 '\n\u3000\u3000第四章\u3000开标、评标和中标\n',
 '',
 '\n\u3000\u3000第五章\u3000法律责任\n',
 '',
 '\n\u3000\u3000第六章\u3000附则\n',
 '',
 '\n第一章\u3000总则\n',
 '\n\u3000\u3000第一条\u3000为了规范招标投标活动，保护国家利益、社会公共利益和招标投标活动当事人的合法权益，提高经济效益，保证项目质量，制定本法。\n\n\u3000\u3000第二条\u3000在中华人民共和国境内进行招标投标活动，适用本法。\n\n\u3000\u3000第三条\u3000在中华人民共和国境内进行下列工程建设项目包括项目的勘察、设计、施工、监理以及与工程建设有关的重要设备、材料等的采购，必须进行招标:\n\n\u3000\u3000（一）大型基础设施、公用事业等关系社会公共利益、公众安全的项目；\n\n\u3000\u3000（二）全部或者部分使用国有资金投资或者国家融资的项目；\n\n\u3000\u3000（三）使用国际组织或者外国政府贷款、援助资金的项目。\n\n\u3000\u3000前款所列项目的具体范围和规模标准，由国务院发展计划部门会同国务院有关部门制订，报国务院批准。\n\n\u3000\u3000法律或者国务院对必须进行招标的其他项目的范围有规定的，依照其规定。\n\n\u3000\u3000第四条\u3000任何单位和个人不得将依法必须进行招标的项目化整为零或者以其他任何方式规避招标。\n\n\u3000\u3000第五条\u3000招标投标活动应当遵循公开、公平、公正和诚实信用的原则。\n\n\u3